# Import Important Libraries

In [3]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Load Data

In [4]:
url = "C:/Users/USER/my_workspace/synapse/loan-default-risk/data/raw/customer_banking_profile.csv"
df = pd.read_csv(url) 

### Explore Dataset

In [5]:
print(df.info())
print(df.describe())
print(df.shape)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4346 entries, 0 to 4345
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   customerid                  4346 non-null   object 
 1   birthdate                   4346 non-null   object 
 2   bank_account_type           4346 non-null   object 
 3   longitude_gps               4346 non-null   float64
 4   latitude_gps                4346 non-null   float64
 5   bank_name_clients           4346 non-null   object 
 6   bank_branch_clients         51 non-null     object 
 7   employment_status_clients   3698 non-null   object 
 8   level_of_education_clients  587 non-null    object 
dtypes: float64(2), object(7)
memory usage: 305.7+ KB
None
       longitude_gps  latitude_gps
count    4346.000000   4346.000000
mean        4.626189      7.251356
std         7.184832      3.055052
min      -118.247009    -33.868818
25%         3.354953      6.47061

In [6]:
# Check uniqueness 
df['customerid'].nunique()

4334

In [7]:
# Check nulls
df['customerid'].isnull().sum()

np.int64(0)

In [8]:
# Check duplicates
df['customerid'].duplicated().sum()
# Drop duplicates
df = df.drop_duplicates(subset=['customerid'])
print(df.info())
print(df.shape)

<class 'pandas.core.frame.DataFrame'>
Index: 4334 entries, 0 to 4345
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   customerid                  4334 non-null   object 
 1   birthdate                   4334 non-null   object 
 2   bank_account_type           4334 non-null   object 
 3   longitude_gps               4334 non-null   float64
 4   latitude_gps                4334 non-null   float64
 5   bank_name_clients           4334 non-null   object 
 6   bank_branch_clients         51 non-null     object 
 7   employment_status_clients   3686 non-null   object 
 8   level_of_education_clients  586 non-null    object 
dtypes: float64(2), object(7)
memory usage: 338.6+ KB
None
(4334, 9)


In [9]:
# Drop columns
df = df.drop(['bank_branch_clients', 'level_of_education_clients'], axis=1)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 4334 entries, 0 to 4345
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerid                 4334 non-null   object 
 1   birthdate                  4334 non-null   object 
 2   bank_account_type          4334 non-null   object 
 3   longitude_gps              4334 non-null   float64
 4   latitude_gps               4334 non-null   float64
 5   bank_name_clients          4334 non-null   object 
 6   employment_status_clients  3686 non-null   object 
dtypes: float64(2), object(5)
memory usage: 270.9+ KB
None


In [10]:
# Fill nulls in employment_status
df['employment_status_clients'].fillna('Unknown', inplace=True)

C:\Users\USER\AppData\Local\Temp\ipykernel_12708\2236659453.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df['employment_status_clients'].fillna('Unknown', inplace=True)


In [11]:
# Convert birthdate to age
df['birthdate'] = pd.to_datetime(df['birthdate'])
df['age'] = (pd.Timestamp.now() - df['birthdate']).dt.days // 365
df = df.drop('birthdate', axis=1)
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 4334 entries, 0 to 4345
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerid                 4334 non-null   object 
 1   bank_account_type          4334 non-null   object 
 2   longitude_gps              4334 non-null   float64
 3   latitude_gps               4334 non-null   float64
 4   bank_name_clients          4334 non-null   object 
 5   employment_status_clients  4334 non-null   object 
 6   age                        4334 non-null   int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 270.9+ KB
None


In [13]:
# Load Nigeria States
states = gpd.read_file("C:/Users/USER/my_workspace/synapse/loan-default-risk/data/geospatial/gadm41_NGA_1.shp")

In [14]:
# convert customer data to GeoDataFrame
geometry = [Point(xy) for xy in zip(df.longitude_gps, df.latitude_gps)]
gdf = gpd.GeoDataFrame(df, geometry=geometry, crs="EPSG:4326")

In [15]:
# assign state using spatial join
gdf = gpd.sjoin(
    gdf,
    states[['NAME_1', 'geometry']],
    how='left',
    predicate='within'
)


In [16]:
# clean column
gdf.rename(columns={'NAME_1': 'state'}, inplace=True)

In [17]:
# save
gdf.drop(columns='geometry').to_csv(
    "customer_banking_profile_cleaned.csv",
    index=False
)

In [19]:
# Sanity check
gdf['state'].value_counts().head()


state
Lagos                        1924
Federal Capital Territory     353
Oyo                           352
Ogun                          336
Rivers                        175
Name: count, dtype: int64

In [20]:
print(df)

                            customerid bank_account_type  longitude_gps  \
0     8a858e135cb22031015cbafc76964ebd           Savings       3.319219   
1     8a858e275c7ea5ec015c82482d7c3996           Savings       3.325598   
2     8a858e5b5bd99460015bdc95cd485634           Savings       5.746100   
3     8a858efd5ca70688015cabd1f1e94b55           Savings       3.362850   
4     8a858e785acd3412015acd48f4920d04           Savings       8.455332   
...                                ...               ...            ...   
4341  8a858f155554552501555588ca2b3b40             Other       3.236753   
4342  8a858fc65cf978f4015cf97cee3a02ce           Savings       7.013750   
4343  8a858f4f5b66de3a015b66fc83c61902           Savings       6.295530   
4344  8aaae7a74400b28201441c8b62514150           Savings       3.354206   
4345  8a85896653e2e18b0153e69c1b90265c           Savings       6.661014   

      latitude_gps bank_name_clients employment_status_clients  age  
0         6.528604           

In [21]:
# Inspect final dataframe
print(df.info())

<class 'pandas.core.frame.DataFrame'>
Index: 4334 entries, 0 to 4345
Data columns (total 7 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   customerid                 4334 non-null   object 
 1   bank_account_type          4334 non-null   object 
 2   longitude_gps              4334 non-null   float64
 3   latitude_gps               4334 non-null   float64
 4   bank_name_clients          4334 non-null   object 
 5   employment_status_clients  4334 non-null   object 
 6   age                        4334 non-null   int64  
dtypes: float64(2), int64(1), object(4)
memory usage: 270.9+ KB
None
